In [2]:
import time
import torch
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
from data.gene_graphs import GeneManiaGraph
from scipy import sparse
import joblib 
import sklearn.cluster
import scipy.cluster
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import ward, fcluster
from sklearn.cluster import AgglomerativeClustering


In [3]:
adj = torch.FloatTensor([[1, 1, 0], [1, 1, 0], [0, 0, 1]])
print(adj)

tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 1.]])


In [4]:
x = torch.FloatTensor([[2, 3, 4], [5, 6, 7]])
print(x)
print(x.shape)

tensor([[2., 3., 4.],
        [5., 6., 7.]])
torch.Size([2, 3])


In [5]:
print(x.size(-1))
new_x = x[0].view(1, x.size(-1))
print(new_x)

3
tensor([[2., 3., 4.]])


In [6]:
res =  new_x * adj
print(res)

tensor([[2., 3., 0.],
        [2., 3., 0.],
        [0., 0., 4.]])


In [7]:
res.max(dim=1)[0]

tensor([3., 3., 4.])

In [8]:
adj = torch.FloatTensor([[1, 1, 0], [1, 1, 0], [0, 0, 1]])
print(adj)

tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 1.]])


In [9]:
x = torch.FloatTensor([[2, 3, 4], [5, 6, 7]])
print(x)
print(x.shape)

tensor([[2., 3., 4.],
        [5., 6., 7.]])
torch.Size([2, 3])


In [10]:
print(x.size(-1))
new_x = x[0].view(1, x.size(-1))
print(new_x)

3
tensor([[2., 3., 4.]])


In [11]:
res =  new_x * adj
print(res)

tensor([[2., 3., 0.],
        [2., 3., 0.],
        [0., 0., 4.]])


In [12]:
res.max(dim=1)[0]

tensor([3., 3., 4.])

In [13]:
gene_graph = GeneManiaGraph()
neighbors = gene_graph.bfs_sample_neighbors("RPL13", 100)
adj = np.asarray(nx.to_numpy_matrix(neighbors))

Torrent name: genemania.pkl, Size: 9.61MB


In [14]:
sparseadj = sparse.csr_matrix(adj)


In [15]:
np.fill_diagonal(adj, 1.)  # TODO: Hummm, think it's a 0.
D = adj.sum(axis=1)


In [16]:
D_inv = np.diag(1. / np.sqrt(D))


In [21]:
D_inv_sparse = sparse.csr_matrix(D_inv)
adj_sparse = sparse.csr_matrix(adj)

In [22]:
norm_transform = D_inv_sparse.dot(adj_sparse).dot(D_inv_sparse)


In [23]:
norm_transform

<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1294 stored elements in Compressed Sparse Row format>

In [24]:
x = torch.FloatTensor(np.random.rand(10, 640, 16000))
print(x.shape)

print(x[0])
print(x[0].shape)


torch.Size([10, 640, 16000])
tensor([[0.4713, 0.6697, 0.8128,  ..., 0.4968, 0.1299, 0.9680],
        [0.2080, 0.8876, 0.0029,  ..., 0.1910, 0.2735, 0.1622],
        [0.9127, 0.6155, 0.8032,  ..., 0.3604, 0.8360, 0.5748],
        ...,
        [0.9560, 0.4817, 0.1504,  ..., 0.3776, 0.4315, 0.0741],
        [0.3368, 0.1659, 0.8138,  ..., 0.7461, 0.0477, 0.0287],
        [0.2886, 0.0969, 0.4913,  ..., 0.0821, 0.9740, 0.7731]])
torch.Size([640, 16000])


In [18]:
print(x.size(-1))
new_x = x[0]
print(new_x)
print(new_x.shape)

16000
tensor([[4.5850e-01, 5.7025e-02, 6.1852e-01,  ..., 7.2319e-02, 7.9605e-01,
         8.7304e-01],
        [4.0838e-01, 2.7077e-01, 4.0068e-02,  ..., 1.6847e-01, 9.3962e-01,
         9.2008e-01],
        [7.9781e-01, 1.5439e-01, 3.1015e-01,  ..., 5.5421e-02, 5.6407e-02,
         5.9987e-01],
        ...,
        [4.3337e-01, 4.6144e-01, 7.0950e-01,  ..., 1.5952e-02, 8.7084e-01,
         1.2626e-01],
        [7.3853e-01, 6.2751e-04, 2.0656e-01,  ..., 3.4056e-01, 2.1334e-01,
         3.6693e-01],
        [2.5889e-01, 1.1269e-01, 9.1408e-01,  ..., 3.7513e-01, 6.0452e-01,
         1.0884e-01]])
torch.Size([640, 16000])


In [21]:
temp = []
start = time.time()
#new_x = new_x.cuda(1)
for i in range(640):
    temp.append((new_x[i].view(1, 16000, 1) * adj).max(dim=1)[0].cpu())
print(time.time() - start)
res = torch.stack(temp).cuda()

TypeError: mul(): argument 'other' (position 1) must be Tensor, not numpy.ndarray

In [20]:
res.shape

torch.Size([3, 3])

In [ ]:
(new_x[0].view(1, 16000, 1).cuda() * adj.cuda()).shape

In [ ]:
%timeit -n 1 -r 2 new_x[0].view(1, 16000, 1).cuda() * adj.cuda()

In [ ]:

            # import pdb; pdb.set_trace()
            # start = time.time()
            # max_value = (x.view(-1, x.size(-1), 1) * adj).max(dim=1)[0]



In [ ]:
        # import pdb; pdb.set_trace()
        # start = time.time()
        #
        # for i in range(x.shape[0]):
        #     temp[i] = (x[i] * adj).max(dim=1)[0]
        # print(str(time.time() - start))
        # max_value = temp
        # import pdb; pdb.set_trace()


In [ ]:
            # temp = []
            # x_cpu = x.view(-1, x.size(-1)).cpu()
            # start = time.time()
            # print("starting")
            # import pdb; pdb.set_trace()
            # 
            # for row in adj.t():
            #     neighbors = row.nonzero().view(-1)
            #     temp.append(x_cpu[:, neighbors].max(dim=1)[0].cuda())
            # print(time.time() - start)
            # max_value = torch.stack(temp).t()
            # 
            # what we used to do: it runs out of memeory
            # max_value = (x.view(-1, x.size(-1), 1) * adj.cuda()).max(dim=1)[0]


In [25]:
ids = sklearn.cluster.AgglomerativeClustering(n_clusters=50, affinity='euclidean', memory='.cache', connectivity=(adj > 0.).astype(bool), compute_full_tree='auto', linkage='ward').fit_predict(adj.astype("float32"))


In [26]:
ids

array([26, 45, 49, 33, 28, 18, 11, 42, 30,  3,  2,  2, 16, 34,  9, 14, 31,
        5, 41, 17,  4,  8, 32, 40, 12, 34, 12, 27, 19, 43, 10, 40, 44, 27,
       18, 39, 10, 47,  7, 22,  1, 13, 48, 10,  3, 35, 14, 12, 21, 16, 35,
       19, 24,  4, 28, 15,  7, 38,  5,  5, 23,  9,  1, 13, 33, 18,  7, 29,
       25, 37,  1, 25,  6,  4,  7, 19,  8, 16, 46,  0, 28,  6, 21, 11, 20,
       23, 17, 37, 30, 24, 12, 22,  8, 46, 36, 15,  0, 29, 26, 20])

In [1]:
temp = adj[:5000, :5000]

NameError: name 'adj' is not defined

In [29]:
sparse_temp = sparse.csr_matrix(temp)

In [30]:
%timeit sklearn.cluster.AgglomerativeClustering(n_clusters=2000, affinity='euclidean', memory='.cache', connectivity=(sparse_temp > 0.).astype(bool), compute_full_tree='auto', linkage='ward').fit_predict(temp.astype("float32"))


281 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
